In [ ]:
import pandas as pd
import torch
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import os
from google.colab import files

!pip3 install kobert-transformers

# review_data_fin.csv, BERTClassifier.py, best_kobert_model_of_shop_review.pth, Classifying_Polarity.py 파일 colab의 contents 폴더에 업로드
# 업로드한 파일의 경로 지정
csv_file = "/content/review_data_fin.csv"  # 업로드된 CSV 파일 경로
bert_classifier_file = "/content/BERTClassifier.py"  # 업로드된 BERTClassifier.py 경로
model_file = "/content/best_kobert_model_of_shop_review.pth"  # 업로드된 모델 파일 경로
Classifying_polarity_file = "/content/Classifying_Polarity.py"  # 업로드된 Classifying_Polarity.py 경로
# CSV 파일 로드
rev = pd.read_csv(csv_file, index_col=0)


from classifying_Polarity import Classifying_Polarity # classifying_Polarity.py가 Colab 환경에 업로드된 후 임포트
from BERTClassifier import BERTClassifier  # BERTClassifier.py가 Colab 환경에 업로드된 후 임포트
from kobert_transformers import get_kobert_model, get_tokenizer

bert_model = get_kobert_model()
model = BERTClassifier(bert=bert_model)
model.eval()  # 평가 모드로 설정
tokenizer = get_tokenizer()

# GPU 또는 CPU 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

# 전체 리뷰 데이터를 리스트로 변환
new_content = rev['content'].to_list()

# 결과를 저장할 리스트
pol_list = []

# 멀티스레딩을 사용하여 모델 예측 실행
with ThreadPoolExecutor() as executor:
    # 진행 상황 표시를 위한 tqdm 설정
    with tqdm(total=len(new_content), desc="Processing Reviews") as pbar:
        
        future_to_text = {executor.submit(Classifying_Polarity.analyze_content, text, model, tokenizer, device=device): text for text in new_content}

        # 작업이 완료될 때마다 결과 처리
        for future in as_completed(future_to_text):
            text = future_to_text[future]
            try:
                result = future.result()
                pol_list.append(result)  # 결과 리스트에 추가
            except Exception as exc:
                print(f"Generated an exception: {exc} for text: {text}")
            finally:
                pbar.update(1)  # 진행 상황 업데이트

# 결과를 데이터프레임에 저장
rev['polarity'] = pol_list

# Step 5: Colab 로컬 파일로 다운로드
rev.to_csv('/content/review_data_fin_polarity.csv', index=False)

# 파일 다운로드
files.download('/content/review_data_fin_polarity.csv')